In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json
import undetected_chromedriver as uc

In [ ]:
def scroll_method():
    # This method is used to scroll the page
    last_height = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        else:
            last_height = new_height


def detect_type_article(link: str) -> str:
    # This function is used to detect the type of article
    if 'liveblog' in link:
        return 'liveblog'
    elif 'blogs.timesofisrael.com' in link:
        return 'blog'
    elif 'https://jewishchronicle.timesofisrael.com/' in link:
        return 'jewishchronicle'
    else:
        return 'article'

def collect_page_titles() -> list[dict]:
    driver.get('https://www.timesofisrael.com')
    scroll_method()

    result = []
    h = driver.find_elements(By.XPATH, '//div[@class="headline"]/a')
    for link in h:
        result.append({
            'title': link.text, 
            'link': link.get_attribute('href'),
            'type': detect_type_article(link.get_attribute('href')),
        })
    return result

def scrape_liveblog():
    # This function is used to scrape the live blog
    pass

def scrape_article(scrape_object: dict) -> dict:
    driver.get(scrape_object['link'])
    scroll_method()
    try:
        title = driver.find_element(By.XPATH, '//h1[@class="headline"]').text
        content = driver.find_element(By.XPATH, '//div[@class="the-content"]').text
        date = driver.find_element(By.XPATH, '//span[@class="date"]').text
        return {
            'title': title,
            'date': date,
            'link': scrape_object['link'],
            'type': scrape_object['type'],
            'content': content
        }
    except Exception as e:
        print(f"Error: {e}")
        return None


def scrape_blog():
    pass



#driver  = webdriver.Chrome()
#titles_to_scrape: list[dict] = collect_page_titles()
#with open('result.jsonl', 'w') as f:
#    for i in titles_to_scrape:
#        f.write(json.dumps(i, ensure_ascii=False) + '\n')
#
#    f.close()
    
#driver.quit()

In [6]:
def scrape_article(scrape_object: dict) -> dict:
    driver.get(scrape_object['link'])
    scroll_method()
    try:
        title = driver.find_element(By.XPATH, '//h1[@class="headline"]').text
        content = driver.find_element(By.XPATH, '//div[@class="the-content"]').text
        date = driver.find_element(By.XPATH, '//span[@class="date"]').text
        return {
            'title': title,
            'date': date,
            'link': scrape_object['link'],
            'type': scrape_object['type'],
            'content': content
        }
    except Exception as e:
        print(f"Error: {e}")
        return None


def scraper_wrapper(scrape_object: dict) -> dict:
    driver.get(scrape_object['link'])
    scroll_method()



dicts = []
with open('result.jsonl', 'r') as f:
    for line in f:
        dicts.append(json.loads(line))
    f.close()

driver  = uc.Chrome(headless=False,use_subprocess=False)
for i in dicts:
    if i['type'] == 'article':
        print(i['link'])
        with open('scaped_articles.jsonl', 'a') as f:    
            x = scrape_article(i)
            f.write(json.dumps(x, ensure_ascii=False) + '\n')
            f.close()
driver.quit()


https://www.timesofisrael.com/israel-claims-its-promoting-palestinian-emigration-from-gaza-so-why-are-so-few-leaving/
https://www.timesofisrael.com/two-idf-soldiers-killed-six-wounded-in-southern-gaza-fighting/
https://www.timesofisrael.com/israeli-plan-to-initially-only-feed-60-of-gazans-as-they-endure-extreme-deprivation/
https://www.timesofisrael.com/leading-agency-shuts-its-gaza-soup-kitchens-amid-continued-israeli-aid-ban/
https://www.timesofisrael.com/eu-to-review-trade-ties-with-israel-following-criticism-of-conduct-in-war-in-gaza/
Error: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//h1[@class="headline"]"}
  (Session info: chrome=136.0.7103.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x655085a1178a <unknown>
#1 0x6550854b40a0 <unknown>
#2 0x6550855059b0 <unknown>
#3 0x655085505ba1 <unknown>
#4 0x655085553ea4 <unknown>
#5 

```   author = driver.find_element(By.XPATH, '//span[@class="byline"]/a').text
```
Note to self that author can have several <a> elements if more than one author